# Intro

Data Source:
https://re.jrc.ec.europa.eu/pvg_tools/en/tools.html

https://joint-research-centre.ec.europa.eu/photovoltaic-geographical-information-system-pvgis/getting-started-pvgis/api-non-interactive-service_en#ref-4-python-wrap-for-api-calls

Info
- Entry point: PVGIS 5.3: https://re.jrc.ec.europa.eu/api/v5_3/tool_name?param1=value1&param2=value2&...
- variables:
- tool_name: PVcalc, SHScalc, MRcalc, DRcalc, seriescalc, tmy, printhorizon. 
ours is HRcalc?? HR?

- param1=value1, param2=value2, ...: input parameters of the tool with their corresponding values concatenated in a query string format.

- API functions accept GET method only
- API calls have a rate limit of 30 calls/second per IP address.

- outputformat=json JSON output.

Our plan:
- take radiation data from this site to feed into our model / similar model to ours
- process radiation data available from site,
- feed to model, 
- predict PV production,
- return prediction

# Fetching data

In [60]:
import sys
import urllib.parse
import requests
import os, csv, json, requests
import glob
import pandas as pd
import numpy as np
from datetime import date

BASE_URI = 'https://re.jrc.ec.europa.eu/api/seriescalc' # for the hourly radiation

lat = 42.6977 # Berlin coords
lon = 23.3219
startyear = 2021
endyear = 2023

params = {
    "lat": lat,
    "lon": lon,
    "startyear": startyear,
    "endyear": endyear,
    "pvcalculation": 1, # If "0" outputs only solar radiation calculations,
        # if "1" outputs the estimation of hourly PV production as well.
    "peakpower": 1, # need if inputting pvcalc as 1
        # our microgrid 'rated output power' is 86.4kWp
    "loss": 14,
    "components": 1, # If "1" outputs beam, diffuse and reflected radiation components.
        #Otherwise, it outputs only global values.
    'optimalangles': 1,
    "outputformat": "json",
    # "browser": # unsure if needed
}

# don't know if we need the below. try without first
# params = "&".join([f'{key}={value}' for key, value in pvgis_params.items()])
# url_seriescalc = f'{url_base}&{params}'

result = requests.get(BASE_URI, params=params).json()

In [61]:
result

{'inputs': {'location': {'latitude': 42.6977,
   'longitude': 23.3219,
   'elevation': 558.0},
  'meteo_data': {'radiation_db': 'PVGIS-SARAH3',
   'meteo_db': 'ERA5',
   'year_min': 2021,
   'year_max': 2023,
   'use_horizon': True,
   'horizon_db': None,
   'horizon_data': 'DEM-calculated'},
  'mounting_system': {'fixed': {'slope': {'value': 36, 'optimal': True},
    'azimuth': {'value': -6, 'optimal': True},
    'type': 'free-standing'}},
  'pv_module': {'technology': 'c-Si', 'peak_power': 1.0, 'system_loss': 14.0}},
 'outputs': {'hourly': [{'time': '20210101:0010',
    'P': 0.0,
    'Gb(i)': 0.0,
    'Gd(i)': 0.0,
    'Gr(i)': 0.0,
    'H_sun': 0.0,
    'T2m': -2.52,
    'WS10m': 1.45,
    'Int': 0.0},
   {'time': '20210101:0110',
    'P': 0.0,
    'Gb(i)': 0.0,
    'Gd(i)': 0.0,
    'Gr(i)': 0.0,
    'H_sun': 0.0,
    'T2m': -2.74,
    'WS10m': 1.45,
    'Int': 0.0},
   {'time': '20210101:0210',
    'P': 0.0,
    'Gb(i)': 0.0,
    'Gd(i)': 0.0,
    'Gr(i)': 0.0,
    'H_sun': 0.0,
 

# Exporting output into CSV

In [62]:
import pandas as pd

# Extract the hourly data from the API response
hourly_data = result['outputs']['hourly']

# Convert the hourly data into a DataFrame
df = pd.DataFrame(hourly_data)

# Define the path to save the CSV
csv_path = '/Users/niki/code/Niki827/watt_squad/eu_sidehustle/aggregating_preprocessing/predictions/02_fit_for_pandas/Berlin.csv'

# Save the DataFrame as a CSV
df.to_csv(csv_path, index=False)

print(f"CSV file saved at: {csv_path}")

CSV file saved at: /Users/niki/code/Niki827/watt_squad/eu_sidehustle/aggregating_preprocessing/predictions/02_fit_for_pandas/Berlin.csv


In [73]:
df['H_sun'].describe()

count    26280.000000
mean        14.868180
std         19.789109
min          0.000000
25%          0.000000
50%          0.000000
75%         27.030000
max         70.390000
Name: H_sun, dtype: float64

# Aggregating the data

In [14]:
# Define the path to the data folder
data_path = '/Users/niki/code/Niki827/watt_squad/eu_sidehustle/aggregating_preprocessing/predictions/02_fit_for_pandas'

# List of cities (corresponding to CSV filenames)
city = 'Berlin'

In [63]:
# Define the aggregation function
def aggregate_data(df):
    # Convert 'time' to datetime and create 'date' column
    df['time'] = pd.to_datetime(df['time'], format='%Y%m%d:%H%M')  # Adjust format if needed
    df['date'] = df['time'].dt.date  # Extract the date for aggregation

    # Convert 'P' to 'pv_output' in kWh and sum daily
    df['pv_output'] = (df['P'] / 1000) * 1  # Convert W to kWh assuming 1-hour intervals
    daily_pv_output = df.groupby('date')['pv_output'].sum().reset_index()

    # Convert irradiance values (Gb(i), Gd(i), Gr(i)) to energy in kWh/m^2 and sum daily
    for col, new_col in {'Gb(i)': 'direct_irradiance', 'Gd(i)': 'diffuse_irradiance', 'Gr(i)': 'reflected_irradiance'}.items():
        df[new_col] = (df[col] * 1) / 1000  # Multiply by time interval (1 hour) and convert to kWh/m^2
        df[new_col] = df[new_col].fillna(0)  # Handle NaN values

    daily_irradiance = df.groupby('date')[['direct_irradiance', 'diffuse_irradiance', 'reflected_irradiance']].sum().reset_index()

    # Average daily values for sun height, temperature, and wind speed
    daily_averages = df.groupby('date')[['H_sun', 'T2m', 'WS10m']].mean().reset_index()
    daily_averages.rename(columns={
        'H_sun': 'sun_height',
        'T2m': 'temp',
        'WS10m': 'wind_speed'
    }, inplace=True)

    # Merge all daily data
    final_daily_data = pd.merge(daily_pv_output, daily_irradiance, on='date')
    final_daily_data = pd.merge(final_daily_data, daily_averages, on='date')

    # Select the required columns
    final_daily_data = final_daily_data[['date', 'pv_output', 'direct_irradiance', 'diffuse_irradiance',
                                         'reflected_irradiance', 'sun_height', 'temp', 'wind_speed']]

    return final_daily_data

In [65]:
aggregated_df = aggregate_data(df)

## running some checks

In [59]:
# Ensure the 'time' column is parsed as datetime if it's not already
df['time'] = pd.to_datetime(df['time'], format='%Y%m%d:%H%M')

# Filter the DataFrame for rows in January 2021
january_2021 = df[(df['time'].dt.year == 2023) & (df['time'].dt.month == 3)]

# Sum the pv_output column for this filtered DataFrame
total_pv_output = january_2021['P'].sum()

print(f"Total PV output for January 2021: {total_pv_output}")

Total PV output for January 2021: 135455.99


# Preprocessing the data

In [68]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import numpy as np


# Define the periodicities function
def add_periodicities(df):
    # Transforming column to datetime
    df['date'] = pd.to_datetime(df['date'])

    # Calculate the day of the year
    df['day_of_year'] = df['date'].dt.dayofyear

    # Total days in the year (account for leap years)
    df['days_in_year'] = df['date'].dt.is_leap_year.apply(lambda x: 366 if x else 365)

    # Compute the cyclic features
    df['year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / df['days_in_year'])
    df['year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / df['days_in_year'])

    # Drop irrelevant columns
    df.drop(columns=['date', 'day_of_year', 'days_in_year'], inplace=True)

    return df

# Define the preprocessing function
def preprocess_data(df):
    # Define the columns to scale
    scale_columns = [
        'direct_irradiance',
        'diffuse_irradiance',
        'reflected_irradiance',
        'sun_height',
        'temp',
        'wind_speed'
    ]

    # Initialize the MinMaxScaler
    scaler = MinMaxScaler()

    # Apply the scaler only to the specified columns
    df[scale_columns] = scaler.fit_transform(df[scale_columns])

    # Add periodicity features
    df = add_periodicities(df)

    return df

In [69]:
preprocessed_df = preprocess_data(aggregated_df)

In [70]:
preprocessed_df

,pv_output,direct_irradiance,diffuse_irradiance,reflected_irradiance,sun_height,temp,wind_speed,year_sin,year_cos
0,4.41120,0.651195,0.172267,0.237844,0.029372,0.398888,0.212890,1.721336e-02,0.999852
1,4.42896,0.650898,0.184596,0.240233,0.030733,0.378107,0.251370,3.442161e-02,0.999407
2,1.93357,0.205744,0.194823,0.123025,0.032266,0.427103,0.297355,5.161967e-02,0.998667
3,0.54822,0.005225,0.118586,0.051317,0.033865,0.440309,0.252442,6.880243e-02,0.997630
4,4.13977,0.572899,0.214248,0.229271,0.035614,0.416292,0.337980,8.596480e-02,0.996298
...,...,...,...,...,...,...,...,...,...
1090,4.05569,0.581201,0.195969,0.220759,0.023973,0.467802,0.196688,-6.880243e-02,0.997630
1091,4.05634,0.587216,0.196420,0.223209,0.024772,0.433175,0.256612,-5.161967e-02,0.998667
1092,4.08063,0.592236,0.200142,0.225658,0.025593,0.441922,0.153919,-3.442161e-02,0.999407
1093,3.85244,0.541790,0.204021,0.215983,0.026673,0.431582,0.160234,-1.721336e-02,0.999852
